In [1]:
#라이브러리 설치
import torch
import ultralytics
from ultralytics import YOLO
import json
import os
import numpy as np
import pandas as pd
import yaml
import shutil
from random import sample
import glob
import ast

In [2]:
# Torch 버전 확인 1.6이상이여야 함
print(torch.__version__)

1.13.1+cu116


In [3]:
# Python 버전확인 3.7이상
import sys
print(sys.version)

3.7.15 (default, Nov 24 2022, 18:44:54) [MSC v.1916 64 bit (AMD64)]


In [4]:
#객체 탐지할 클래스명 리스트로 작성
classes=['halibut'] #두개 클래스로 Yolo test진행

In [5]:
#원본 data 경로
path='C:/Users/user/Desktop/flex_data/flex_data'
# new data path->Train,Vaild,Test로 분할해서 분석을 진행해야하기 때문에 설정해줌
save_path='C:/Users/user/Desktop/flex_data'

# datasets 폴더 생성

## train, valid, test 폴더 각각 생성

In [6]:
def createFolder(save_dir):
    try:
        for tvt in ['train', 'test', 'valid']:
            for IL in ['images', 'labels']:
                dir = save_dir + '/' + 'TVT_data_yolo' + '/' + tvt + '/' + IL 
                if not os.path.exists(dir):
                    os.makedirs(dir)
                else:
                    print('Folder has already been created')
    except OSError:
        print('Error: Creating directory. ' + save_dir)

In [7]:
createFolder(save_dir=save_path)

# TVT 데이터 분할

In [8]:
# # 파일 리스트화
img_list=[]
lab_list=[]
img_path=glob.glob(path+'/'+'images'+'/*')
lab_path=glob.glob(path+'/'+'annotations'+'/*')
for i in range(len(img_path)):
    img_path[i]=img_path[i].replace('\\','/')
    lab_path[i]=lab_path[i].replace('\\','/')
img_list.extend(img_path)
lab_list.extend(lab_path)

In [9]:
print(len(img_list)) #이미지 파일명
print(len(lab_list)) #라벨링 파일명
print(img_list[0])
print(lab_list[0])

100
100
C:/Users/user/Desktop/flex_data/flex_data/images/SA12_T01_FR_000002.jpg
C:/Users/user/Desktop/flex_data/flex_data/annotations/SA12_T01_FR_000002.json


In [10]:
def get_tv_idx(tl, p = 0.8, m = 0.5):
    total_idx = range(tl)
    train_idx = sample(total_idx, int(tl * p)) #전체에서 80% 데이터 추출
    temp_idx = set(total_idx) - set(train_idx) #전체 인덱스값에서 트레인 데이터에 해당하는 인덱스 제거
    valid_idx = sample(temp_idx, int(len(temp_idx) * m)) #남은 인덱스값에서 50%추출->전체데이터에서 10%추출
    test_idx = set(temp_idx) - set(valid_idx) #전체에서 10%데이터 추출

    return train_idx, valid_idx, list(test_idx)

In [11]:
train_idx, valid_idx, test_idx = get_tv_idx(len(img_list))
print(len(train_idx))
print(len(valid_idx))
print(len(test_idx))

80
10
10


In [12]:
# Yolo용 dataset옮겨주는 경로
save_path='C:/Users/user/Desktop/flex_data/TVT_data_yolo'

In [13]:
# Train, Valid, Test 경로
train_path=path+'/'+'train'
valid_path=path+'/'+'valid'
test_path=path+'/'+'test'

# TVT별 이미지, 라벨링 데이터 경로 복사 및 이동

In [14]:
train_img_list = [img_list[idx] for idx in train_idx]
train_lab_list= [lab_list[idx] for idx in train_idx]
valid_img_list = [img_list[idx] for idx in valid_idx]
valid_lab_list = [lab_list[idx] for idx in valid_idx]
test_img_list = [img_list[idx] for idx in test_idx]
test_lab_list = [lab_list[idx] for idx in test_idx]

In [15]:
# train
for i in train_img_list:
    shutil.copy(i,save_path+'/'+'train'+'/'+'images') # image copy
for j in train_lab_list:
    shutil.copy(j,save_path+'/'+'train'+'/'+'labels') #label copy 

#valid
for i in valid_img_list:
    shutil.copy(i,save_path+'/'+'valid'+'/'+'images') # image copy
for j in valid_lab_list:
    shutil.copy(j,save_path+'/'+'valid'+'/'+'labels') #label copy 

# test
for i in test_img_list:
    shutil.copy(i,save_path+'/'+'test'+'/'+'images') # image copy
for j in test_lab_list:
    shutil.copy(j,save_path+'/'+'test'+'/'+'labels') #label copy 

In [16]:
# Train Data 이미지 경로 복사

In [17]:
train_path=save_path+'/'+'train'

In [18]:
train_img_list=[] #이미지 파일명 리스트

train_img=glob.glob(train_path+'/'+'images'+'/*')
for j in range(len(train_img)):
    train_img[j]=train_img[j].replace('\\','/') #경로 \\부분 /로 수정
    train_img_list.append(train_img[j]) # 각 이미지 경로 저장

In [19]:
# Vaild Data 이미지 경로 복사
valid_path=save_path+'/'+'valid'

In [20]:
valid_img_list=[] #이미지 파일명 리스트

valid_img=glob.glob(valid_path+'/'+'images'+'/*')

for j in range(len(valid_img)):
    valid_img[j]=valid_img[j].replace('\\','/') #경로 \\부분 /로 수정
    valid_img_list.append(valid_img[j]) # 각 이미지 경로 저장

In [21]:
# test Data 이미지 경로 복사
test_path=save_path+'/'+'test'

In [22]:
test_img_list=[] #이미지 파일명 리스트

test_img=glob.glob(test_path+'/'+'images'+'/*')
for j in range(len(test_img)):
    test_img[j]=test_img[j].replace('\\','/') #경로 \\부분 /로 수정
    test_img_list.append(test_img[j]) # 각 이미지 경로 저장

In [23]:
# Train image 경로, Valid image 경로 txt 파일 저장
with open(train_path+'/'+'train.txt','w') as f:
    f.write('\n'.join(train_img_list)+'\n')
    
with open(valid_path+'/'+'valid.txt','w') as f:
    f.write('\n'.join(valid_img_list)+'\n')

with open(test_path+'/'+'test.txt','w') as f:
    f.write('\n'.join(test_img_list)+'\n')

# yaml 파일 만들기

In [24]:
# yaml file 만들기 경로는 코드가 구축된 폴더로 구성하기

data={
    'train':train_path+'/'+'train.txt',
    'val':valid_path+'/'+'valid.txt',
    'test':test_path+'/'+'test.txt',
    'nc':len(classes),
    'names':['halibut']
}

# 최종적으로 생성된 yaml파일을 확인하여 names value값이 리스트형태로 잘 작성되었는지만 확인 해야함

# yaml 파일 저장->YOLO 모델 수행 시 필수 사항임

In [25]:
with open('C:/Users/user/Desktop/project/yolov8/data.yaml', 'w') as f:
    yaml.dump(data, f)

# 인공지능 학습용 데이터 라벨링 파일->Yolov8 dataformat으로 변경

## Train 라벨링 데이터 파일명 경로 복사

In [28]:
train_lab_list=[] #라벨링 파일명 리스트


train_lab=glob.glob(train_path+'/'+'labels'+'/*')
for j in range(len(train_lab)):
    train_lab[j]=train_lab[j].replace('\\','/') #경로 \\부분 /로 수정
    train_lab_list.append(train_lab[j]) # 각 이미지 경로 저장

print(len(train_lab_list))

80


# Valid 라벨링 데이터 이미지 파일명 경로 복사

In [29]:
valid_lab_list=[] #라벨링 파일명 리스트


valid_lab=glob.glob(valid_path+'/'+'labels'+'/*')
for j in range(len(valid_lab)):
    valid_lab[j]=valid_lab[j].replace('\\','/') #경로 \\부분 /로 수정
    valid_lab_list.append(valid_lab[j]) # 각 이미지 경로 저장

print(len(valid_lab_list))

10


# Test 라벨링 데이터 이미지 파일명 경로 복사

In [30]:
test_lab_list=[] #라벨링 파일명 리스트


test_lab=glob.glob(test_path+'/'+'labels'+'/*')
for j in range(len(test_lab)):
    test_lab[j]=test_lab[j].replace('\\','/') #경로 \\부분 /로 수정
    test_lab_list.append(test_lab[j]) # 각 이미지 경로 저장

print(len(test_lab_list))

10


# 라벨링 데이터 파일 전처리 Train

In [31]:
# sample용 train data labeling file 하나 open

In [32]:
def convert_label(label_list): #저장할 라벨링 파일 리스트, 저장 경로, Train,vaild,Test중 선택하여 mode작성
    for i in label_list:
        # 각 이미지 당 하나의 파일을 생성해야하기 때문에 labels, bboxs 매 사진마다 초기화 필요
        labels = [] #class값 저장
        bboxs = [] #x_center, y_center, width, height순으로
        with open(i,"r",encoding='UTF-8') as f:
            jsonfile=json.load(f) #json file불러오기
            
        box_objects = jsonfile['Annotations'] #어노테이션 정보 저장
        image_size = np.asarray(ast.literal_eval(jsonfile['Info']['resolution'])) #이미지 해상도
        w, h = [int(i) for i in image_size] #weight, height
        
        for obj in box_objects:
            if obj['annotation_type']=='rect': #Bounding Box 버전
                labels.append(classes.index(obj['class_code'])) #라벨에 클래스 넘버 추가하기
                bboxs.append(ast.literal_eval(obj['rect.points']))
            elif obj['annotation_type']=='polygon':
                labels.append(classes.index(obj['class_code']))
                poly_list=ast.literal_eval(obj['polygon.points'])
                xlist=poly_list[0]
                ylist=poly_list[1]
                xmin,xmax,ymin,ymax=min(xlist),max(xlist),min(ylist),max(ylist)
                bboxs.append([xmin,ymin,xmax,ymax]) #bbox는 x최소, y최소, x최대, y최대로 이루어져있음
            else:
                print('annotation type must be "rect" or "polygon"')
        #좌표값 수정
        bboxs = np.asarray(bboxs, dtype=np.float64)
        try:
            bboxs[:, [0,2]] /= w #넓이 
            bboxs[:, [1,3]] /= h #높이
        except: 
            pass
        
        labels = np.array(labels).reshape(len(labels), 1) #라벨값 이어붙이기
        box_cxy = (bboxs[:, 2:] + bboxs[:, :2])/2.0 #xcenter,ycenter
        box_wh = bboxs[:, 2:] - bboxs[:, :2] #width, height
        new_box = np.concatenate((box_cxy, box_wh), axis=1) #합치기
        annot_out = np.concatenate((labels, new_box), axis=1) #라벨값과 합치기
        
        with open(i.split('.')[0] + '.txt', 'w') as f:
            for line in annot_out:
                line = np.asarray(line, dtype=str)
                line[0] = line[0].split('.')[0]
                f.write(' '.join(line) + '\n')    

In [33]:
convert_label(train_lab_list) #train data yolo 라벨링 전용 데이터 생성

In [34]:
convert_label(valid_lab_list)

In [35]:
convert_label(test_lab_list)

In [36]:
model = YOLO(model='yolov8n.pt')
device = 'cuda'
model.to(device)
model.train(data='C:/Users/user/Desktop/project/yolov8/data.yaml', epochs=1, patience=100, batch=2, imgsz=640, pretrained=False, optimizer='SGD', conf=0.001, lr0=0.0037)
model.val()

New https://pypi.org/project/ultralytics/8.0.166 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.112  Python-3.7.15 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Users/user/Desktop/project/yolov8/data.yaml, epochs=1, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x000001DA05CE4688>
fitness: 0.0074177290831388965
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([  0.0059725])
names: {0: 'halibut'}
plot: True
results_dict: {'metrics/precision(B)': 0.009666666666666667, 'metrics/recall(B)': 0.9666666666666667, 'metrics/mAP50(B)': 0.020424865033674147, 'metrics/mAP50-95(B)': 0.005972491755301647, 'fitness': 0.0074177290831388965}
save_dir: WindowsPath('runs/detect/val')
speed: {'preprocess': 2.900242805480957, 'inference': 26.89945697784424, 'loss': 0.0, 'postprocess': 3.3000707626342773}